In [1]:
with open("input.txt", "r") as file: 
    data = file.read()

## Part 1

In [2]:
import re

In [3]:
def split(data): 
    """
    Split the batch file by passport, where passport is a key:value 
    dictionary
    """
    passports = []
    
    for passport in re.split("\n{2,}", data): 
        fields = {}
        for field in re.split("\s+", passport.strip()):
            name, value = field.split(":")
            assert name not in fields
            fields[name] = value
        passports.append(fields)
    return passports

assert isinstance(split(data)[0], dict)

def validate(passport):
    """
    A passport is valid if and only if it has all the required 
    fields
    """
    required = ["byr","iyr","eyr","hgt","hcl","ecl","pid"]
    return all(field in passport for field in required)

assert validate({}) == False
assert validate({key:None for key in ["byr","iyr","eyr","hgt","hcl","ecl","pid"]}) == True

def run(data): 
    return sum(validate(passport) for passport in split(data))

run(data)

260

## Part 2

In [4]:
class Validator: 
    def validate(self, value):
        raise NotImplementedError

class Integer(Validator):
    def validate(self, value):
        return re.match("^\d+$", value) is not None
    
assert Integer().validate("2014") == True
assert Integer().validate("2.95") == False
    
class Min(Validator):
    def __init__(self, minimum, strict=False): 
        self.minimum = minimum
        self.strict  = strict
        
    def validate(self, value): 
        if self.strict: 
            return self.minimum < float(value)
        return self.minimum <= float(value)
    
assert Min(1).validate("9") == True
assert Min(1).validate("0") == False

class Max(Validator):
    def __init__(self, maximum, strict=False): 
        self.maximum = maximum
        self.strict  = strict
        
    def validate(self, value): 
        if self.strict: 
            return self.maximum > float(value)
        return self.maximum >= float(value)

assert Max(1).validate("9") == False
assert Max(1).validate("0") == True

class Pattern(Validator): 
    def __init__(self, pattern):
        self.pattern = pattern
        
    def validate(self, value):
        return re.match(self.pattern, value) is not None
    
class Any(Validator):
    def __init__(self, children):
        self.children = children
        
    def validate(self, value):
        for child in self.children: 
            if child.validate(value):
                return True
        return False
    
class All(Validator):
    def __init__(self, children):
        self.children = children
        
    def validate(self, value):
        for child in self.children: 
            if not child.validate(value):
                return False
        return True
    
class Sanitizer(Validator):
    def __init__(self, callback, child): 
        self.callback = callback
        self.child = child
        
    def validate(self, value):
        return self.child.validate(self.callback(value))
    
def validate(obj, schema):
    for field in schema: 
        #if the field is required, check it exists
        if field.get("required", False) and field["name"] not in obj:
            return False
        
        #run all the validators for this field
        for validator in field.get("validators", []): 
            if not validator.validate(obj[field["name"]]): 
                return False
            
    #it passed all tests, it must be valid
    return True

In [5]:
schema = [
    {
         "name":"byr", 
         "required":True, 
         "validators":[Integer(), Min(1920), Max(2002)]
    },
    {
        "name":"iyr", 
        "required":True, 
        "validators":[Integer(), Min(2010), Max(2020)]},
    {
        "name":"eyr", 
        "required":True, 
        "validators":[Integer(), Min(2020), Max(2030)]},
    {
        "name":"hgt", 
        "required":True, 
        "validators":[Any([
            All([
                Pattern("\d{3}cm"), 
                Sanitizer(lambda x: float(x[:-2]), Min(150)),
                Sanitizer(lambda x: float(x[:-2]), Max(193))
            ]),
            All([
                Pattern("\d{2}in"), 
                Sanitizer(lambda x: float(x[:-2]), Min(59)),
                Sanitizer(lambda x: float(x[:-2]), Max(76))
            ])
        ])]},
    {
        "name":"hcl", 
        "required":True, 
        "validators":[Pattern("^#[0-9a-f]{6}$")]
    },
    {
        "name":"ecl",
        "required":True, 
        "validators":[Pattern("^amb|blu|brn|gry|grn|hzl|oth$")]
    },
    {
        "name":"pid",
        "required":True,
        "validators":[Pattern("^\d{9}$")]
    },
    {
        "name":"cid", 
        "required":False
    }
]

In [6]:
def run(data): 
    return sum(validate(passport, schema) for passport in split(data))

run(data)

153